In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
from pandas import read_csv
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
from xgboost import plot_importance
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error , make_scorer
from math import sqrt
# from sklearn.impute import SimpleImputer
import torch
device = torch.device("cuda")

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self, classification=False, class_number=1):
        super().__init__()
        input_size = 7

        self.BN_before = torch.nn.BatchNorm1d(10)
        self.rnn=torch.nn.LSTM(
            input_size= input_size,
            hidden_size=128,
            num_layers=2,
            batch_first=True
        )
        self.linear1 = torch.nn.Linear(128, 1)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(128, class_number)

    def forward(self,x):
        out = x
#         out = self.BN_before(out)
        output,(h_n,c_n) = self.rnn(out)
        out = output[:,-1,:] 
        out = self.linear1(out)
#         out = self.relu(out)
#         out = self.linear2(out)
        
        return out
    
    def getFeature(self, x):
        x = torch.from_numpy(x).to(device)
        output,(h_n,c_n) = net.rnn(x)
        out = output[:,-1,:].squeeze(0).cpu().detach().numpy()
        return out
    

In [ ]:
#################################################################
net = LSTM(False, 1).to(device)
net.load_state_dict(torch.load('./lstm_10.pkl')) # correct here
#[1, 10, 7] -> [128]

In [ ]:

train_epoch = 50
batch_size = 32
learning_rate = 0.0001
output_normalized = True
output_delta = True



In [ ]:
train_dataframe = read_csv('../input/cs410ei339-fall18-stock-price-prediction/train_data.csv', engine='python')

train_dataframe = train_dataframe.drop(columns=['Unnamed: 0', 'Time'])
train_dataset = train_dataframe.values

X, delta_Y, original_Y = [],[], []
for i in range(len(train_dataset)-30):
    if (i+1) % 100000 == 0:
        print("{}/{}".format(i, len(train_dataset)-30))
    tmp = train_dataset[i:i+30]
    if tmp[0,0] != tmp[-1,0]:
        continue
    tmp = tmp[:, 1:].astype(np.float32)
    delta_ave_midPrice = np.average(tmp[10:, 0]) - tmp[9, 0]
    X.append(tmp[:10])
    delta_Y.append(delta_ave_midPrice)
    original_Y.append(np.average(tmp[10:, 0]))
X, delta_Y, original_Y = np.array(X), np.array(delta_Y), np.array(original_Y)


In [ ]:
X[0]

In [ ]:
if not output_delta:
    Y = original_Y
else:
    Y = delta_Y

In [ ]:
def feature_engin(X):
    XX = X.reshape((-1,70),order="F")

    #the features:
    # [0:10]: mid price
    # [10:20]: Last price
    # [20:30]:Volume
    # [30:40]:BidPrice1
    # [40:50]:BidVolume
    # [50:60]:AskPrice
    # [60:70]:AskVolume
    # [70:80]: first order difference of midprice, the first one is constant 0
    # [80:90]: first order average of midprice, the first one is same as the first element
    # [90:100]: second order average: the average of the first average
    # [100:110]: third order average and their difference: (with first five arbitrary)
    # [110:120]: difference of second order average(with three arbitrary)
    # [120:130]: the arbrartry average of the difference of the second order average
    # [130:140]: the difference between BidPrice and Ask Price
    # [140:150]: the difference between AskVolume and BidVolume
    # [150:160]: the log sum of the above two things
    # [160:170]: the average and time difference of the log sum
    # [170:180]: the average and time difference of the difference between BidPrice and AskPrice
    # [180:190]: the average and time difference of the difference between AskVolume and BidVolume
    # [190:200]: arbitrary

    #[70:80]add the first order difference
    XX = np.append(XX,np.zeros((XX.shape[0],1)), axis = 1)
    for i in range(9):
        XX = np.append(XX,XX[:,i+1].reshape(-1,1) - XX[:,i].reshape(-1,1),axis = 1)

    #[80:90] first order average
    XX = np.append(XX,XX[:,10].reshape(-1,1), axis = 1) #arbitarly choose
    for i in range(9):
        XX = np.append(XX,(XX[:,i+1].reshape(-1,1) + XX[:,i].reshape(-1,1))/2,axis = 1)

    #[90:100] second order average
        #three arbitrary combination of the features
    XX = np.append(XX,-0.5*XX[:,9].reshape(-1,1),axis = 1)
    XX = np.append(XX,XX[:,20].reshape(-1,1) - XX[:,10].reshape(-1,1),axis = 1)
    for i in range(8):
        XX = np.append(XX,(XX[:,i+82].reshape(-1,1) + XX[:,i+80].reshape(-1,1))/2,axis = 1)

    #[100:110] 5 arbitary + 5 third order average and their difference :
    XX = np.append(XX,( (XX[:,0] + XX[:,1] + XX[:,2]+XX[:,3]+XX[:,4]).reshape(-1,1))/5,axis = 1)
    XX = np.append(XX,( (XX[:,5] + XX[:,6] + XX[:,7]+XX[:,8]+XX[:,9]).reshape(-1,1))/5,axis = 1)
    XX = np.append(XX, (XX[:,101] / XX[:,100] - 1).reshape(-1,1),axis = 1)
    XX = np.append(XX,( (XX[:,10] + XX[:,11] + XX[:,12]+XX[:,13]+XX[:,14]).reshape(-1,1))/5,axis = 1)
    XX = np.append(XX,( (XX[:,15] + XX[:,16] + XX[:,17]+XX[:,18]+XX[:,19]).reshape(-1,1))/5,axis = 1)
    for i in range(3):
        XX = np.append(XX,(XX[:,i+93].reshape(-1,1) + XX[:,i+97].reshape(-1,1))/2,axis = 1)
    for i in range(2):
        XX = np.append(XX,(XX[:,i+106].reshape(-1,1) - XX[:,i+105].reshape(-1,1)),axis = 1)

    #[110:120]: difference of second order average
    XX = XX = np.append(XX,(XX[:,104] - XX[:,101]).reshape(-1,1),axis = 1)
    for i in range(7):
        XX = np.append(XX,((XX[:,i+93]- XX[:,i+90]).reshape(-1,1)),axis = 1)
    XX = np.append(XX,( (XX[:,110] + XX[:,111] + XX[:,112]+XX[:,113]+XX[:,114]).reshape(-1,1))/5,axis = 1)
    XX = np.append(XX,( (XX[:,115] + XX[:,116] + XX[:,117]+XX[:,118]).reshape(-1,1))/4,axis = 1)
    # [120:130]: the arbitrary average of the difference of the second order average
    for i in range(10):
        XX = np.append(XX,((XX[:,2*i+100]+ XX[:,i+119]).reshape(-1,1))/2,axis = 1)

    # [130:140]: the difference between BidPrice and AskPrice
    for i in range(10):
        XX = np.append(XX,((XX[:,i+50]- XX[:,i+30]).reshape(-1,1)),axis = 1)
    # [140:150]: the difference between AskVolume and BidVolume
    for i in range(10):
        XX = np.append(XX,((XX[:,i+60]- XX[:,i+40]).reshape(-1,1)),axis = 1)
    # [150:160]: the log sum of the above two things
    for i in range(10):
        XX = np.append(XX,( (XX[:,i+140]+ np.log(XX[:,i+130]+0.001)).reshape(-1,1)),axis = 1)

    # [160:170]: the average and time difference of the log sum
    for i in range(5):
        XX = np.append(XX,((XX[:,2*i+150]+ XX[:,2*i+151]).reshape(-1,1))/2,axis = 1)
    for i in range(3):
        XX = np.append(XX,((XX[:,i+162]- XX[:,i+160]).reshape(-1,1)),axis = 1)
    XX = np.append(XX,(((XX[:,155]+ XX[:,156]+ XX[:,157]+ XX[:,158]+ XX[:,159])/5).reshape(-1,1)),axis = 1)
    XX = np.append(XX,((XX[:,168]- (XX[:,150]+ XX[:,151]+ XX[:,152]+ XX[:,153]+ XX[:,154])/5 ).reshape(-1,1)),axis = 1)

    # [170:180]: the average and time difference of the difference between BidPrice and AskPrice
    for i in range(5):
        XX = np.append(XX,((XX[:,2*i+130]+ XX[:,2*i+131]).reshape(-1,1))/2,axis = 1)
    for i in range(3):
        XX = np.append(XX,((XX[:,i+172]- XX[:,i+170]).reshape(-1,1)),axis = 1)
    XX = np.append(XX,(((XX[:,135]+ XX[:,136]+ XX[:,137]+ XX[:,138]+ XX[:,139])/5).reshape(-1,1)),axis = 1)
    XX = np.append(XX,((XX[:,178]- (XX[:,130]+ XX[:,131]+ XX[:,132]+ XX[:,133]+ XX[:,134])/5 ).reshape(-1,1)),axis = 1)

    # [180:190]:  the average and time difference of the difference between AskVolume and BidVolume 
    for i in range(5):
        XX = np.append(XX,((XX[:,2*i+140]+ XX[:,2*i+141]).reshape(-1,1))/2,axis = 1)
    for i in range(3):
        XX = np.append(XX,((XX[:,i+182]- XX[:,i+180]).reshape(-1,1)),axis = 1)
    XX = np.append(XX,(((XX[:,145]+ XX[:,146]+ XX[:,147]+ XX[:,148]+ XX[:,149])/5).reshape(-1,1)),axis = 1)
    XX = np.append(XX,((XX[:,188]- (XX[:,140]+ XX[:,141]+ XX[:,142]+ XX[:,143]+ XX[:,144])/5 ).reshape(-1,1)),axis = 1)

    # [190:200]: arbitrary

#     num_iter = math.floor(X.shape[0]/128)
#     s = []
#     print(X.shape)
#     for i in range(num_iter):
#         s.append(net.getFeature(X[i*128:(i+1)*128]))
#     s = np.array(s).reshape(-1, 128)
#     s2 = net.getFeature(X[num_iter*128:])
#     s_final  = np.vstack((s, s2))

#     XX_final = np.hstack((XX, s_final))
#     XX = XX_final
#     print(XX.shape)
    return XX


In [ ]:
XX = feature_engin(X)
XX[0]

In [ ]:
import math
X_mean = np.mean(X, axis=(1, ))[:, np.newaxis, :]
X_std = np.std(X, axis=(1, ))[:, np.newaxis, :] + 0.01
X_normalized = (X-X_mean) / X_std

num_iter = math.floor(X_normalized.shape[0]/128)
s = []
for i in range(num_iter):
    s.append(net.getFeature(X_normalized[i*128:(i+1)*128]))

In [ ]:
s = np.array(s).reshape(-1, 128)
s2 = net.getFeature(X_normalized[num_iter*128:])
s_final  = np.vstack((s, s2))

XX_final = np.hstack((XX, s_final))
print(XX_final.shape)


In [ ]:
XX = XX_final

In [ ]:
XX = X.reshape(-1,70)

In [ ]:
seed = 7
test_size = 0.33
train_number = int(0.8 * len(XX))
train_X, train_Y = XX[:], Y[:]
print(train_number)
print(XX.shape)
print(train_X.shape)
val_X, val_Y =  XX[:], Y[:]
# train_X = pd.DataFrame(train_X,columns=["t"+str(i) for i in range(190)])
# # train_Y = pd.DataFrame(train_Y,columns=[i for i in range(190)])
# val_X = pd.DataFrame(val_X,columns=["f"+str(i) for i in range(190)])
# print(train_X.columns)
# print(val_X.columns)
# val_Y = pd.DataFrame(val_Y,columns=[i for i in range(190)])
# train_X, train_Y = X[:train_number].reshape((-1,70)), Y[:train_number]
# val_X, val_Y =  X[train_number:].reshape((-1,70)), original_Y[train_number:]
# val_X = val_X[:, :70]
# train_X = train_X[:, :70]

In [ ]:
# my_pipeline = Pipeline([('imputer', Imputer()), ('xgbrg', XGBRegressor())])
# param_grid = {
#     "xgbrg__n_estimators": [200],
#     "xgbrg__learning_rate": [0.01],
#     "xgbrg__max_depth" :[3],#3
#     "xgbrg__min_child_weight" :[6],#6
#     "xgbrg__gamma" :[0], #0
#     "xgbrg__subsample" :[0.7],
#     "xgbrg__colsample_bytree" :[0.9],
#     "xgbrg__scale_pos_weight" :[1],
#     "xgbrg__reg_alpha" :[0.01]
# }

# # train_X_matrix = train_X.as_matrix()
# # train_Y_matrix = train_Y.as_matrix()
# # val_X_matrix = val_X.as_matrix()
# # val_Y_matrix = val_Y.as_matrix()


# fit_params = {"xgbrg__eval_set": [(val_X, val_Y)], 
#               "xgbrg__early_stopping_rounds": 10, 
#               "xgbrg__verbose": False} 
# scorer = make_scorer(lambda val, pre: sqrt(mean_squared_error(val, pre)) , greater_is_better= False)
# searchCV = GridSearchCV(my_pipeline, cv=5, param_grid=param_grid, fit_params=fit_params, n_jobs=-1 , scoring=scorer)
# searchCV.fit(train_X, train_Y)  

first search, step 2 max_depth: 3, min child_weight 5, second search max_depth:3 min_child_weight:6

gamma tried 0, 0.05, 0.1 0.2, 0.3 best 0

subsample  0.7 cosample_bytree 0.9


In [ ]:
# print(searchCV.best_params_ )
# print(searchCV.cv_results_['mean_train_score'])
# print(searchCV.cv_results_['mean_test_score'])

In [ ]:
model = XGBRegressor(n_estimators=100)
# model = XGBRegressor()
model.fit(train_X,train_Y)

In [ ]:
print(train_X.shape)

In [ ]:
y_pred = model.predict(val_X)
print(y_pred.shape)
print(val_Y.shape)
accuracy = sqrt( mean_squared_error(val_Y, y_pred))
accuracy

In [ ]:
# accuracy = sqrt( mean_squared_error(val_Y, y_pred))
# accuracy

In [ ]:
net = LSTM(False, 1).to(torch.device("cuda"))
net.load_state_dict(torch.load('../input/lstmpara/lstm_10.pkl')) # correct here

In [ ]:
# load the dataset
test_dataframe = read_csv('../input/cs410ei339-fall18-stock-price-prediction/test_data.csv', engine='python')

test_dataframe = test_dataframe.drop(columns=['Date', 'Unnamed: 0', 'Time'])
test_dataset = test_dataframe.values
test_reshape = test_dataset.reshape(1000, 10, 7).astype(np.float32)#.astype(np.float64)

# 
XX = feature_engin(test_reshape)
X = test_reshape

X_mean = np.mean(X, axis=(1, ))[:, np.newaxis, :]
X_std = np.std(X, axis=(1, ))[:, np.newaxis, :] + 0.01
X_normalized = (X-X_mean) / X_std

num_iter = math.floor(X_normalized.shape[0]/128)
s = []
print(X.shape)
for i in range(num_iter):
    s.append(net.getFeature(X_normalized[i*128:(i+1)*128]))
s = np.array(s).reshape(-1, 128)
print(s.shape)
s2 = net.getFeature(X_normalized[num_iter*128:])
s_final  = np.vstack((s, s2))

XX_final = np.hstack((XX, s_final))
XX = XX_final
print(XX.shape)
test = XX
print(test.shape)
result = model.predict(test)
for i, item in enumerate(result):
    print("{},{}".format(i+1, float(test_reshape[i, 9, 0]) + float(item)))
    

In [ ]:
  a= 7